<a href="https://colab.research.google.com/github/Yusuf80216/my_notebooks/blob/main/fake_news_classifier(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!gdown 1HtSsLQJ-7JWx4rwKrEQ3ZvMxhg-3P04X

Downloading...
From: https://drive.google.com/uc?id=1HtSsLQJ-7JWx4rwKrEQ3ZvMxhg-3P04X
To: /content/train.csv
100% 98.6M/98.6M [00:00<00:00, 126MB/s]


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv('train.csv')

In [26]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [27]:
df = df.dropna()

In [28]:
X = df.drop('label', axis = 1)
y = df['label']

In [29]:
X.shape

(18285, 4)

In [30]:
y.shape

(18285,)

In [31]:
import tensorflow as tf

In [32]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [33]:
vocab_size = 5000

OneHot Representation

In [34]:
messages = df.copy()

In [35]:
messages.reset_index(inplace = True)

In [36]:
import nltk
import re
from nltk.corpus import stopwords

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
### Data Preprocessing

from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [39]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [41]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
onehot_repr

[[1901, 380, 1274, 3595, 2168, 4632, 1576, 3606, 401, 4431],
 [4075, 322, 3859, 4647, 4996, 4599, 1501],
 [969, 3939, 3421, 1824],
 [2748, 613, 432, 1615, 3252, 2615],
 [4224, 4996, 1796, 2606, 3997, 319, 4996, 2201, 676, 85],
 [890,
  3387,
  3727,
  1074,
  1883,
  469,
  2597,
  648,
  222,
  4132,
  1910,
  2560,
  2863,
  4610,
  1501],
 [3043, 1706, 1532, 2621, 3401, 1795, 4619, 1899, 3938, 1219, 1566],
 [4872, 1536, 3387, 2645, 572, 36, 469, 4298, 3938, 1219, 1566],
 [3356, 2432, 3003, 4046, 3784, 2254, 1498, 2399, 469, 353],
 [438, 1470, 2732, 3951, 4790, 1820, 2951, 2530],
 [3539, 4233, 2078, 2739, 485, 3719, 4700, 2837, 3033, 3005, 338],
 [1615, 4280, 2168, 2254, 469, 572],
 [2026, 3001, 908, 3939, 1251, 3862, 3785, 1790, 3670],
 [4901, 946, 2914, 4278, 1206, 4587, 3757, 3938, 1219, 1566],
 [3744, 1497, 4023, 2960, 1567, 3938, 1219, 1566],
 [2901, 3329, 2681, 4292, 1428, 2240, 3356, 3143, 4675, 2623],
 [4725, 1237, 322],
 [3420, 2642, 2045, 974, 469, 4929, 4121, 1501],
 [1647

Embedding Representation

In [44]:
sent_len = 20

embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_len)
embedded_docs

array([[   0,    0,    0, ..., 3606,  401, 4431],
       [   0,    0,    0, ..., 4996, 4599, 1501],
       [   0,    0,    0, ..., 3939, 3421, 1824],
       ...,
       [   0,    0,    0, ..., 3938, 1219, 1566],
       [   0,    0,    0, ..., 3682, 4190, 2825],
       [   0,    0,    0, ..., 2131, 1388, 1675]], dtype=int32)

In [ ]:
embedding_vector_features = 40

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [46]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [47]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

Model Training

In [49]:
model.fit(X_train, y_train, validation_data = [X_test, y_test], batch_size = 64, epochs = 10)

Epoch 1/10
192/192 [==============================] - 9s 33ms/step - loss: 0.3334 - accuracy: 0.8435 - val_loss: 0.1920 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1375 - accuracy: 0.9450 - val_loss: 0.1952 - val_accuracy: 0.9203
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0948 - accuracy: 0.9646 - val_loss: 0.2073 - val_accuracy: 0.9117
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0664 - accuracy: 0.9772 - val_loss: 0.2504 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0452 - accuracy: 0.9856 - val_loss: 0.3002 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0292 - accuracy: 0.9899 - val_loss: 0.3536 - val_accuracy: 0.9036
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0174 - accuracy: 0.9944 - val_loss: 0.3758 - val_accuracy: 0.9120

Adding Dropout Layers

In [53]:
from tensorflow.keras.layers import Dropout

In [58]:
embedding_vector_features = 40

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.fit(X_train, y_train, validation_data = [X_test, y_test], batch_size = 64, epochs = 10)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.3374 - accuracy: 0.8449 - val_loss: 0.1930 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1467 - accuracy: 0.9411 - val_loss: 0.1886 - val_accuracy: 0.9216
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1078 - accuracy: 0.9595 - val_loss: 0.2036 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0867 - accuracy: 0.9677 - val_loss: 0.2263 - val_accuracy: 0.9127
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0670 - accuracy: 0.9741 - val_loss: 0.2671 - val_accuracy: 0.9143
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0496 - accuracy: 0.9833 - val_loss: 0.2790 - val_accuracy: 0.9150
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0378 - accuracy: 0.9869 - val_loss: 0.3249 - val_accuracy: 0.9152

Prediction and Accuracy

In [62]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 4ms/step
